# Data Manipulation

De in te lezen data is helaas niet publiek toegankelijk. Om deze data toch voor demo doeleinden te kunnen gebruiken heb ik 
middels deze routine de data een klein beetje aangepast. Hierbij is er op gelet om de samenhang van de data in de curves en in de tijd te behouden.


In [4]:
import pandas as pd
import numpy as np
import datetime
from pandas.tseries.offsets import DateOffset           # Gebruikt voor het berekenen van de datum van FRA contracten

import random as rd                                        

In [5]:

def read_data():
    '''
    Lees de marktdata in.    
    Input: 
      -        
    Output: 
      Pandas Dataframe    
    '''            
    try:
    
        # Datum kolommen
        date_cols = ["rate_dt","input_dt","actual_dt",'start_term_dt','maturity_term_dt']
    
        # Omzetten datum kolommen. int_basis en exchange name zijn gewoon string, maar werden niet goed herkend
        df = pd.read_csv(r'data\rates_data_original.csv', 
                         parse_dates = date_cols, 
                         dtype ={"int_basis":'S10', "exchange_name": 'S10'})

        
    except Exception as e:
            print("ERROR: Unable to find or access file:", e)
    
    return df

# Inlezen data
df = read_data()


In [6]:
# Toevoegen van gemiddelde en standaard afwijking 

keys = ['curve_type','rate_name','rate_type','ccy','time_band']
df_gr = df.groupby(keys).agg( {'mid' : ['mean', 'std'] } ) 

# hernoemen output kolommen
df_gr.columns = ['mean', 'std'] 

# Group by levert een multi index. Deze wordt hier weer plat geslagen
df_gr = df_gr.reset_index() 

# Berekenen aanpassing per rate

df_gr['change'] = round(df_gr['std']*(rd.random()-.5),5)

# Left join de change over alle data
dfi = df.merge(df_gr, on = keys, how = "left")

# aanpassen van de data
dfi['bid'] = dfi['bid']+dfi['change']
dfi['offer'] = dfi['bid']+dfi['change']
dfi['mid'] = dfi['bid']+dfi['change']

# Drop werk kolommen
del dfi['change']
del dfi['mean']
del dfi['std']


# Write the changed data to disc
dfi.to_csv(r'data\rates_data.csv',index=False)